In [1]:
%pip install ipythonblocks
%pip install qpsolvers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 2.8 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/bscpnu/Praktikum-Kecerdasan-Komputasional.git

Cloning into 'Praktikum-Kecerdasan-Komputasional'...
remote: Enumerating objects: 494, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 494 (delta 19), reused 24 (delta 8), pack-reused 449 (from 1)
Receiving objects: 100% (494/494), 52.87 MiB | 11.94 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [3]:
%cd "/content/Praktikum-Kecerdasan-Komputasional/Praktikum/2 - Logical Agents"

/content/Praktikum-Kecerdasan-Komputasional/Praktikum/2 - Logical Agents


In [4]:
import sys
sys.path.append("/content/Praktikum-Kecerdasan-Komputasional/Praktikum/2 - Logical Agents")

In [5]:
!ln -s /content/Praktikum-Kecerdasan-Komputasional/aima-data ./aima-data
!ls aima-data

ascii-robotdata1.log  GE-text	     LICENSE.md  orings.txt	 zoo.txt
ascii-robotdata1.txt  gutenberg.txt  MAN	 README.md
EN-text		      iris.csv	     MNIST	 restaurant.csv
FR-text		      iris.txt	     orings.csv  zoo.csv


In [6]:
from utils import *
from logic import *

from notebook import psource

/content/Praktikum-Kecerdasan-Komputasional/Praktikum/2 - Logical Agents/logic.py:682: SyntaxWarning: invalid escape sequence '\s'
  For each literal compute J(l) = \sum{l in clause c} 2^{-|c|}
/content/Praktikum-Kecerdasan-Komputasional/Praktikum/2 - Logical Agents/csp.py:642: SyntaxWarning: invalid escape sequence '\ '
  downs[i]     Number of queens in the \ diagonal


# Logic

Notebook Jupyter ini berfungsi sebagai materi pendukung untuk topik-topik yang dibahas dalam __Bab 6 Logical Agents__, __Bab 7 First-Order Logic__ and __Bab 8 Inference in First-Order Logic__ dari buku *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. Kami menggunakan implementasi dalam modul [logic.py](https://github.com/bscpnu/Praktikum-Kecerdasan-Komputasional/blob/main/Praktikum/2%20-%20Logical%20Agents/logic.py). Lihat [panduan praktikum logic](https://github.com/bscpnu/Praktikum-Kecerdasan-Komputasional/blob/main/Praktikum/2%20-%20Logical%20Agents/2%20Panduan%20Praktikum%20Logic.pdf) untuk petunjuk lebih lanjut.

Mari kita impor terlebih dahulu semua yang ada dalam modul logika.

## CONTENTS
- Logical sentences
    - Expr
    - PropKB
    - Knowledge-based agents
    - Inference in propositional knowledge base
        - Truth table enumeration
        - Proof by resolution
        - Forward and backward chaining
        - DPLL
        - WalkSAT
        - SATPlan
    - FolKB
    - Inference in first order knowledge base
        - Unification
        - Forward chaining algorithm
        - Backward chaining algorithm

## Logical Sentences (Kalimat Logika)

Kelas `Expr` dirancang untuk mepresentasikan berbagai ekspresi matematis. Tipe paling sederhana dari `Expr` adalah sebuah symbol, yang bisa didefinisikan dengan fungsi `Symbol`:

In [7]:
Symbol('x')

# perhatikan penggunaan huruf kapital/tidak di awal kata

x

Atau juga bisa mendefinisikan beberapa symbol sekaligus dengan menggunakan fungsi `symbols`:

In [8]:
(x, y, P, Q, f) = symbols('x, y, P, Q, f')

Kita bisa menggabungkan `Expr` dengan operator infiks dan prefiks Python biasa. Contohnya, begini cara kita membentuk kalimat logika "P dan bukan Q":

In [9]:
P & ~Q

(P & ~Q)



Hal ini bisa dilakukan karena kelas `Expr` melakukan overloading pada operator `&` dengan definisi berikut:

```
def __and__(self, other):
    return Expr('&', self, other)
```

Selain `&`, operator lain juga di-override dengan cara serupa. Setiap `Expr` punya dua bagian penting:

- `op` → operator (selalu berupa string)

- `args` → argumen, yaitu tuple berisi 0 atau lebih ekspresi

maksud dari "ekspresi" di sini bisa berupa objek `Expr` lain, atau angka biasa. kita bisa lihat isi field dari beberapa contoh `Expr`:

In [10]:
sentence = P & ~Q | f

sentence.op
# operator paling atas dari ekspresi

'|'

In [11]:
sentence.args
# mengambil argumen dari ekspresi

((P & ~Q), f)

In [12]:
Q.op

'Q'

In [16]:
P.args
# mengecek argumen dari simbol

()

P = (). karena P berdiri sendiri (tidak punya argumen)

In [17]:
Pxy = P(x, y)
# ekspresi baru, P adalah sebuah predikat dengan argumen x dan y

Pxy.op
# operator utama dari P(x, y) adalah:

'P'

In [18]:
Pxy.args
# argumen dari P(x, y)

(x, y)

⚠️ Note
- Expr tidak mendefinisikan logika (misalnya kebenaran True/False).

- Expr hanya cara untuk merepresentasikan bentuk kalimat logika (seperti tree/AST). [abstract syntax tree](https://en.wikipedia.org/wiki/Abstract_syntax_tree)

- Setiap dari `args` didalam `Expr` bisa berupa either symbol, number, atau `Expr` bersarang, dan kita bisa menumpuk ekspresi sedalam apa pun.


Contoh:

In [19]:
3 * f(x, y) + P(y) / 2 + 1

(((3 * f(x, y)) + (P(y) / 2)) + 1)

## Operators for Constructing Logical Sentences

Tabel berikut menunjukkan operator-operator yang bisa dipakai untuk membentuk kalimat logika.


👀: Perhatikan ada sedikit masalah: kita ingin memakai operator Python untuk menuliskan kalimat logika, supaya kode yang kita tulis (dan hasil di sesi interaktif) tetap sederhana. Masalahnya

Namun, Python **tidak mengizinkan simbol panah implikasi (`==>`)** sebagai operator.  
Sebagai gantinya, kita harus menggunakan notasi yang lebih panjang, yaitu `|'==>'|` alih-alih `==>`. Alternatifnya, kita bisa langsung menggunakan bentuk konstruktor `Expr` yang lebih eksplisit.

| Operasi                  | Notasi Buku | Input Python (Infix) | Output Python | Input dengan `Expr` |
|---------------------------|-------------|----------------------|---------------|---------------------|
| Negasi                    | ¬P          | `~P`                 | `~P`          | `Expr('~', P)`      |
| Konjungsi (And)           | P ∧ Q       | `P & Q`              | `P & Q`       | `Expr('&', P, Q)`   |
| Disjungsi (Or)            | P ∨ Q       | `P | Q`              | `P | Q`       | `Expr('|', P, Q)`   |
| Eksklusif Or (Xor)        | P ≠ Q       | `P ^ Q`              | `P ^ Q`       | `Expr('^', P, Q)`   |
| Implikasi (→)             | P → Q       | `P |'==>'| Q`        | `P ==> Q`     | `Expr('==>', P, Q)` |
| Implikasi Terbalik (←)    | Q ← P       | `Q |'<=='| P`        | `Q <== P`     | `Expr('<==', Q, P)` |
| Ekuivalensi (↔)           | P ↔ Q       | `P |'<=>'| Q`        | `P <=> Q`     | `Expr('<=>', P, Q)` |


Contoh bagaimana kita mendefinisikan sebuah kalimat logika dengan panah implikasi:

In [20]:
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

### `expr`: sebuah shortcut untuk membentuk kalimat

Jika `|'==>'|` terlihat jelek (tampilannya), bisa juga menggunakan fungsi `expr`.

In [21]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

Fungsi `expr` bisa dipakai buat mengubah string menjadi sebuah objek `Expr`. Di dalam string, kita bisa menggunakan operator panah (`==>`, `<==`, `<=>`) seolah-olah dan memang akan diperlakukan itu sebagai operator Python biasa.

Selain itu, perlu diingat bahwa `expr` juga otomatis mendefinisikan simbol apa pun yang muncul, jadi kita tidak perlu mendefinisikannya terlebih dahulu. let say ada simbol baru (misalnya a, b, c), dia otomatis bikin simbolnya tanpa kita harus definisikan dulu.

In [22]:
expr('sqrt(a ** 2 - 4 * b * c)')

sqrt(((a ** 2) - ((4 * b) * c)))

Untuk saat ini, itu saja yang perlu diketahui tentang `expr`. Jika penjelasan lebih detail mengenai bagaimana `expr` diimplementasikan dan bagaimana `|'==>'|` ditangani dapat ditemukan di bagian lampiran (appendix).

## Propositional Knowledge Bases: `PropKB`

Kelas `PropKB` digunakan untuk merepresentasikan **basis pengetahuan (knowledge base / KB)** yang berisi kalimat-kalimat logika proposisional.

Sebelum masuk ke `PropKB`, perhatikan dulu bahwa kelas `KB` memiliki empat metode utama (selain `__init__`). Penting dicatat bahwa metode `ask` sebenarnya hanya memanggil metode `ask_generator`. Artinya, ketika membuat kelas KB sendiri (meskipun kemungkinan besar tidak perlu, karena kelas-kelas sudah disediakan), yang perlu benar-benar diimplementasikan adalah **`ask_generator`**, bukan `ask`.
<br>
<br>
### Kelas `PropKB`

- **`__init__(self, sentence=None)`**  
  Konstruktor `__init__` membuat satu atribut bernama `clauses`, yaitu sebuah list yang berisi semua kalimat di dalam basis pengetahuan.  Setiap kalimat disimpan dalam bentuk *clause* → kalimat yang hanya terdiri dari **literal** dan operator **OR (`∨`)**.

- **`tell(self, sentence)`**
  Ketika ingin menambahkan kalimat ke KB, method `tell` Digunakan untuk menambahkan kalimat baru ke KB.  
  Metode ini akan:
  1. Mengubah kalimat ke bentuk **CNF (Conjunctive Normal Form)**.
  2. Mengekstrak semua *clause*.
  3. Menambahkannya ke atribut `clauses`.

  Jadi, kita tidak perlu  memastikan bahwa kalimat yang kita "tell" ke KB sudah berupa klausa. Kita bisa memberikan kalimat dalam bentuk apa pun, dan proses konversi ke CNF serta penambahan klausa hasil konversi akan ditangani otomatis oleh metode `tell`.

- **`ask_generator(self, query)`**
  Fungsi `ask_generator` dipanggil oleh `ask` untuk menjawab pertanyaan (query).  
  Di dalamnya, ia memanggil fungsi `tt_entails`, yang mengembalikan:
  - `True` jika basis pengetahuan **mengimplikasikan** query (KB ⊨ query).  
  - `False` jika tidak.  

  ✒️ Basically, Entailment itu artinya pengetahuan yang ada di KB menjamin kebenaran suatu kalimat α.

  Hasil dari `ask_generator`adalah:
  - `{}` (dictionary kosong) → jika query benar.  
  - `None` → jika query salah.  

  Mungkin agak aneh, karena biasanya kita expect hasil yang berupa `True` atau `False`. Namun, tujuannya ini dibuat adalah agar konsisten dengan **logika orde pertama (FOL)**, di mana `ask_generator` seharusnya mengembalikan semua **substitusi** variabel yang membuat query benar.  Dalam logika proposisional tidak ada variabel, jadi yang dikembalikan cukup `{}` (menandakan “ada solusi, tapi kosong”).

  (～￣▽￣)～ Jika lebih suka hasil `True` atau `False`, gunakan metode **`ask_if_true`**. ～(￣▽￣~)


 - **`retract(self, sentence)`**
  Digunakan untuk **menghapus** kalimat dari basis pengetahuan. disini, sama seperti `tell`, kita tidak harus memberikan *clause* langsung. Cukup berikan kalimat dalam bentuk apa pun → metode ini akan otomatis mengubahnya ke bentuk *clause* lalu menghapusnya dari `clauses`.

## Wumpus World KB
contoh penggunaan `PropKB` (Propositional Knowledge Base) dalam Wumpus World "sentences mentioned in `section 7.4.3`."

In [23]:
wumpus_kb = PropKB()
# tempat untuk menyimpan semua pengetahuan proposisional tentang dunia Wumpus.

mendefinisikan simbol-simbol yang kita gunakan di klausa kita<br/>
$P_{x, y}$ True jika ada *pit* (lubang) di kotak `[x, y]`.<br/>
$B_{x, y}$ True jika ada *breeze* (angin) di kotak `[x, y]`.<br/>



In [24]:
# Contoh deklarasi:
P11, P12, P21, P22, P31, B11, B21 = expr('P11, P12, P21, P22, P31, B11, B21')

# sekarang kita punya simbol logika proposisional untuk merepresentasikan kondisi dunia.

dari `section 7.4.3`. Kita akan bilang <br/>
"Tidak ada lubang di `[1,1]`".

In [25]:
wumpus_kb.tell(~P11)

**aturan umum di Wumpus World**

Sebuah kotak akan terdapat *breeze* jika dan hanya jika ada *pit* di kotak tetangganya. Hal ini sebenarnya harus dituliskan untuk setiap kotak, tapi untuk sekarang kita hanya menuliskan kotak-kotak yang relevan saja

In [26]:
wumpus_kb.tell(B11 | '<=>' | ((P12 | P21)))
# Breeze di (1,1) muncul jika dan hanya jika ada pit di (1,2) atau (2,1):
wumpus_kb.tell(B21 | '<=>' | ((P11 | P22 | P31)))
# Guees it ^_^

[👉 Lihat Jawaban](#jawaban1)
belum selesai




berdasarkan situasi di `Figure 7.3(b)`, sekarang kita masukkan persepsi breeze untuk dua kotak pertama

In [27]:
wumpus_kb.tell(~B11)
# tidak ada breeze di (1,1)
wumpus_kb.tell(B21)
# another one, Guess it 🤔

### Isi knowledge base (clauses)

Kita bisa cek clauses yang kita simpan di `KB` dengan mengakses variabel `clauses`-nya

In [28]:
wumpus_kb.clauses
# kalau kita cek, nanti isinya sudah berupa bentuk CNF (conjunctive normal form)

[~P11,
 (~P12 | B11),
 (~P21 | B11),
 (P12 | P21 | ~B11),
 (~P11 | B21),
 (~P22 | B21),
 (~P31 | B21),
 (P11 | P22 | P31 | ~B21),
 ~B11,
 B21]

Jadi bisa dilihat bahwa kalimat awal yang kita `tell` tadi otomatis diubah ke bentuk klausa logika proposisional.

Kita melihat bahwa ekivalensi $B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ ecara otomatis dikonversi menjadi dua implikasi yang kemudian diubah lagi menjadi CNF yang disimpan di dalam `KB`.<br/>
$B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ dipecah menjadi $B_{1, 1} \implies (P_{1, 2} \lor P_{2, 1})$ dan $B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$.<br/>
$B_{1, 1} \implies (P_{1, 2} \lor P_{2, 1})$ dikonversi menjadi $P_{1, 2} \lor P_{2, 1} \lor \neg B_{1, 1}$.<br/>
$B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$ dikonversi menjadi $\neg (P_{1, 2} \lor P_{2, 1}) \lor B_{1, 1}$ yang kemudian menjadi $(\neg P_{1, 2} \lor B_{1, 1}) \land (\neg P_{2, 1} \lor B_{1, 1})$ setelah menerapkan hukum De Morgan dan mendistribusikan disjungsi.<br/>
$B_{2, 1} \iff (P_{1, 1} \lor P_{2, 2} \lor P_{3, 2})$ dikonversi dengan cara yang serupa.



## Knowledge based agents (Agen Berbasis Pengetahuan)

Sebuah agen berbasis pengetahuan adalah agen generik sederhana yang menyimpan dan mengelola basis pengetahuan (knowledge base / KB).
Basis pengetahuan ini dapat berisi pengetahuan awal (background knowledge).

Tujuan dari agen KB adalah menyediakan abstraksi untuk manipulasi knowledge base, serta menjadi kelas dasar untuk agen-agen lain yang bekerja dengan knowledge base.


Ketika menerima percept, agen akan menambahkan percept tersebut ke dalam `KB`. Agen lalu bertanya pada `KB` untuk menentukan aksi terbaik. Agen lalu bertanya pada `KB` untuk menentukan aksi terbaik. Setelah melakukan aksi, agen akan memberi tahu `KB` bahwa aksi tersebut sudah dilakukan.



Implementasi `KB-Agent` dituliskan dalam kelas `KB_AgentProgram`, yang merupakan turunan dari kelas `KB`:


Let's have a look 👀

In [29]:
psource(KBAgentProgram)

Sesuai namanya
- `make_percept_sentence` → membuat kalimat logika orde pertama (FOL) tentang percept yang diterima agen.
- `make_action_query` → menanyakan pada `KB` aksi apa yang harus dilakukan. Dan,
- `make_action_sentence` → memberi tahu `KB` aksi yang sudah dilakukan agen.

## Inference in Propositional Knowledge Base
Pada bagian ini kita akan melihat dua algoritma untuk mengecek apakah suatu kalimat $\alpha$ diimplikasikan oleh `KB`. Tujuan kita adalah menentukan apakah $\text{KB} \vDash \alpha$ untuk suatu kalimat $\alpha$.

### Truth Table Enumeration
Metode ini menggunakan pendekatan *model-checking* yang sesuai namanya akan mengenumerasi semua kemungkinan model di mana `KB` benar, lalu memeriksa apakah $\alpha$ juga benar pada model-model tersebut.

Langkah-langkahnya adalah:
1. Mendaftar semua simbol proposisional dalam `KB` (𝑛 simbol).
2. Enumerasi semua $2^{n}$ kemungkinan nilai kebenaran (truth assignments). Contoh: ada 2 simbol: P dan Q. Maka ada $2^2 = 4$ model (kemungkinan)
3. Cek apakah untuk semua model yang membuat KB benar, $\alpha$ juga benar.

*Notes 📃 <br>
Model = sebuah assignment (penugasan) nilai benar/salah untuk setiap simbol proposisional.*

In [30]:
psource(tt_check_all)

pada dasarnya algoritma ini membuat setiap baris dari truth table $KB\implies \alpha$, lalu memeriksa apakah semuanya bernilai benar.

Jika symbols sudah didefiniskan, *routine* (fungsi) ini akan secara rekursif membentuk semua kemungkinan kombinasi nilai kebenaran (True/False) untuk simbol-simbol tersebut.

Model-model yang dihasilkan ini sesuai dengan baris pada tabel kebenaran. Dari semua baris itu, hanya baris yang membuat $KB$ bernilai benar yang akan dipertimbangkan. Pada baris-baris tersebut, sistem kemudian memeriksa apakah query (kalimat logis $\alpha$) juga bernilai benar.


`result = pl_true(alpha, model)`.
Singkatnya, `tt_check_all` mengevaluasi ekspresi logika untuk setiap `model`

`pl_true(kb, model) => pl_true(alpha, model)`

yang secara logika equivalent dengan

`pl_true(kb, model) & ~pl_true(alpha, model)`

🎯 Artinya: KB dan negasi dari query tidak boleh konsisten (tidak boleh ada model yang membuat keduanya benar).


Fungsi `tt_entails()` hanya mengekstrak simbol-simbol dari query dan memanggil `tt_check_all()` dengan parameter yang sesuai


In [31]:
psource(tt_entails)

**⚠️ things that important to remember:**

- kb = knowledge base (basis pengetahuan), berisi semua fakta/logika yang sudah kita ketahui.

- alpha = query / kalimat target yang mau kita cek apakah diterima secara logis oleh kb.

Ingat selalu bahwa dua simbol `P` dan `Q`, implikasi P => Q hanya bernilai salah jika `P` `True` dan `Q` `False`. Contoh penggunaan `tt_entails()`:

In [32]:
tt_entails(P & Q, Q)

# `kb = P & Q`. Artinya, syarat dunia (model) yang valid adalah: P= `True` dan Q= `True`.
# `alpha = Q`. Jadi pertanyaannya: apakah Q benar di semua model yang bikin KB benar?

True

P & Q is True only when both P and Q are True. Hence, (P & Q) => Q is True

In [33]:
tt_entails(P | Q, Q)

False

kenapa False?
| P | Q | KB = P∨Q | α = Q | KB ⊨ α ?             |
| - | - | -------- | ----- | -------------------- |
| F | F | F        | F     | KB salah → diabaikan |
| F | T | T        | T     | ✔ α benar            |
| T | F | T        | F     | ✘ α salah            |
| T | T | T        | T     | ✔ α benar            |

Meskipun `KB (P | Q)` benar di 3 baris, tapi pada baris ke-3 "(T,F)", `KB` benar tapi `α` salah

Karena ada setidaknya satu model di mana KB benar tapi α tidak benar, maka `KB` tidak men-entail `α`

🤔 **why is that?**

`P` ∨ `Q` artinya "`P` atau `Q` benar". maka bisa saja "`P` = `True` dan `Q` = `False`."
hasilnya? `KB` tetap benar, tapi Q (α) tidak benar. Karena entailment harus `α` benar di semua model di mana KB benar, maka `P ∨ Q ⊨ Q` adalah `False`


In [34]:
tt_entails(P | Q, P)

False

Jika kita tahu bahwa P | Q adalah true, kita tidak bisa menurunkan nilai kebenaran P maupun Q secara pasti. sehingga (P | Q) => Q bernilai False dan begitu juga (P | Q) => P



In [35]:
(A, B, C, D, E, F, G) = symbols('A, B, C, D, E, F, G')
tt_entails(A & (B | C) & D & E & ~(F | G), A & D & E & ~F & ~G)

True

Kita bisa lihat agar KB benar, A, D, E harus True, F dan G harus False, Tidak ada informasi yang bisa ditarik tentang B atau C

**Kembali ke kasus Wumpus World**

`tt_entails()` menerima `Expr` berupa konjungsi klausa sebagai input, bukan `KB` secara langsung. kita bisa menggunakan metode `ask_if_true()` dari `PropKB`, karena metode ini otomatis melakukan semua konversi yang diperlukan.


Let's check what `wumpus_kb` tells us about $P_{1, 1}$.



In [36]:
wumpus_kb.ask_if_true(~P11), wumpus_kb.ask_if_true(P11)

(True, False)

Pada Figur 7.9, di semua model di mana KB bernilai `True`, $P_{1, 1}$ adalah `False`. Jadi wajar jika `ask_if_true(~P11)` mengembalikan True dan `ask_if_true(P11)` mengembalikan False.

Tetapi juga memunculkan pertanyaan menarik, apa yang terjadi jika α True hanya di sebagian model saja? apakah akan mengembalikan `True` atau `False`?

Dalam kasus ini, α mungkin benar di beberapa model KB, tapi tidak selalu benar di semua model KB. Karena ask_if_true() hanya mengembalikan True jika α diimplikasikan oleh KB (True di semua model KB), maka hasilnya adalah False.

In [37]:
wumpus_kb.ask_if_true(~P22), wumpus_kb.ask_if_true(P22)

(False, False)

### Proof by Resolution
Ingat Selalu bahwa tujuan kita adalah mengecek apakah $\text{KB} \vDash \alpha$, contohnya apakah $\text{KB} \implies \alpha$ benar di setiap model. Misalkan kita ingin memeriksa apakah $P \implies Q$ valid. Caranya adalah dengan memeriksa *satisfiability* (keterpenuhan) dari negasi $\neg (P \implies Q)$, yang dapat ditulis ulang menjadi $P \land \neg Q$. Jika $P \land \neg Q$  *unsatisfiable* (tidak dapat dipenuhi), maka $P \implies Q$ pasti benar di semua model.

Dari sini didapatkan hasil:

> $\text{KB} \vDash \alpha \iff \text{KB} \land \neg \alpha$

Maksudnya?

untuk mengecek bahwa α selalu benar (entail) di semua model di mana `KB` benar, dapat kita lakukan kontradiksi (gabungkan KB dengan ¬α). Kalau $KB \land \neg \alpha$ tidak bisa dipenuhi (*id est*, tidak ada model yang membuat keduanya benar sekaligus), artinya mustahil α salah ketika `KB` benar. Jadi, satu-satunya kemungkinan adalah α selalu benar jika KB benar

Teknik ini disebut *proof by contradiction* (pembuktian dengan kontradiksi), sebuah teknik pembuktian matematika standar. Kita mengasumsikan $\alpha$ salah, lalu menunjukkan bahwa asumsi ini menghasilkan kontradiksi dengan aksioma yang diketahui di $\text{KB}$. Kontradiksi diperoleh dengan melakukan inferensi menggunakan aturan inferensi. Dalam pembuktian ini kita hanya menggunakan satu aturan inferensi, yaitu resolusi yang menyatakan:

$(l_1 \lor \dots \lor l_k) \land (m_1 \lor \dots \lor m_n) \land (l_i \iff \neg m_j) \implies l_1 \lor \dots \lor l_{i - 1} \lor l_{i + 1} \lor \dots \lor l_k \lor m_1 \lor \dots \lor m_{j - 1} \lor m_{j + 1} \lor \dots \lor m_n$

Dengan menerapkan aturan resolusi, kita memperoleh klausa baru yang ditambahkan ke KB. Proses ini terus dilakukan hingga:

- Tidak ada klausa baru yang bisa ditambahkan → artinya $\text{KB} \nvDash \alpha$

- Ada dua klausa yang diselesaikan (resolved) dan menghasilkan *empty clause* (klausa kosong) → artinya $\text{KB} \vDash \alpha$

Klausa kosong ekuivalen dengan False, karena hanya muncul jika kita menggabungkan dua klausa yang saling berlawanan, sepertu $P$ dan $\neg P$. Ini jelas kontradiksi, sebab $P$ dan $\neg P$ tidak bisa benar secara bersamaan.

**✨ The Analogy >_<**

Kita sedang mencari bukti apakah "Garfield punya ekor" di dalam `KB`

- Kalau setelah dicoba semua kombinasi, nggak ada kontradiksi → mungkin Garfield punya ekor atau mungkin tidak. Sehingga, kesimpulannya tidak pasti = $KB \nvDash \alpha$

- Kalau ketemu kondisi "Garfield punya ekor" dan "Garfield tidak punya ekor" muncul barengan. Artinya? **Kontradiksi**, berarti asumsi awal (Garfield tidak punya ekor) salah → kesimpulan: $KB \vDash \alpha$


Namun ada satu hal penting: algoritma yang mengimplementasikan pembuktian dengan resolusi tidak bisa langsung menangani kalimat kompleks. Implikasi dan bi-implikasi harus disederhanakan dulu menjadi klausa-klausa yang lebih sederhana.

Kita sudah tahu bahwa setiap kalimat logika proposisional secara logis ekuivalen dengan konjungsi klausa-klausa. Fakta ini akan kita manfaatkan untuk menyederhanakan kalimat masukan ke dalam *Conjunctive Normal Form* (CNF), yaitu bentuk konjungsi dari disjungsi literal.

$$(A\lor B)\land (\neg B\lor C\lor\neg D)\land (D\lor\neg E)$$
> Bentuk ini sama dengan POS (Product of Sums)



we can see an outline of how the conversion is done:

1. **Ubah bi-implikasi menjadi implikasi**:

$\alpha\iff\beta$ dapat ditulis sebagai $(\alpha\implies\beta)\land(\beta\implies\alpha)$, Termasuk jika ada kalimat majemuk, misalnya:

$\alpha\iff(\beta\lor\gamma)$ bisa ditulis sebagai $(\alpha\implies(\beta\lor\gamma))\land((\beta\lor\gamma)\implies\alpha)$


2. **Ubah implikasi ke bentuk ekuivalen logis**

$\alpha\implies\beta$ dapat ditulis sebagai $\neg\alpha\lor\beta$

3. **Geser negasi ke dalam**

CNF hanya boleh punya literal atomik, jadi negasi tidak boleh ada di luar kalimat majemuk. Untuk itu hukum De Morgan bisa membantu disini

$\neg(\alpha\land\beta)\equiv(\neg\alpha\lor\neg\beta)$

$\neg(\alpha\lor\beta)\equiv(\neg\alpha\land\neg\beta)$

4. **Distribusikan disjungsi (`OR`) ke atas konjungsi (`AND`)**

Karena konjungsi dan disjungsi saling distributif,

> $A \lor (B \land C) \equiv (A \lor B) \land (A \lor C)$

kita distribusikan disjungsi supaya didapat bentuk konjungsi dari disjungsi literal. Tujuannya adalah mengubah ekspresi logika ke bentuk CNF, yaitu bentuk yang lebih "standar", konjungsi dari disjungsi (AND of OR).
Bentuk akhirnya kira-kira seperti:

We need a term of the form

$(\alpha_{1}\lor\alpha_{2}\lor\alpha_{3}...)\land(\beta_{1}\lor\beta_{2}\lor\beta_{3}...)\land(\gamma_{1}\lor\gamma_{2}\lor\gamma_{3}...)\land...$

Fungsi `to_cnf` dipakai untuk menjalankan proses konversi ini menggunakan *helper subroutines*.

In [38]:
psource(to_cnf)

`to_cnf` memanggil tiga *subroutines*.
<br>
- `eliminate_implications` Hilangkan implikasi/bimp (==>, <=>)

- `move_not_inwards` Gerakkan negasi ke dalam (proposisi atomiknya) menggunakan hukum morgan

- `distribute_and_over_or` distribusi disjunctions ke conjunctions.
<br>
detail implementasi:

In [39]:
psource(eliminate_implications)
psource(move_not_inwards)
psource(distribute_and_over_or)

Let's convert some sentences to see how it works


In [40]:
A, B, C, D = expr('A, B, C, D')
to_cnf(A |'<=>'| B)

((A | ~B) & (B | ~A))

In [41]:
to_cnf(A |'<=>'| (B & C))

((A | ~B | ~C) & (B | ~A) & (C | ~A))

In [42]:
to_cnf(A & (B | (C & D)))

(A & (C | B) & (D | B))

In [43]:
to_cnf((A |'<=>'| ~B) |'==>'| (C | ~D))

((B | ~A | C | ~D) & (A | ~A | C | ~D) & (B | ~B | C | ~D) & (A | ~B | C | ~D))

Kita dapat melihat bagaimana fungsi `to_cnf` dipakai dalam permasalahan resolution ini.

In [44]:
psource(pl_resolution)

In [45]:
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)

(True, False)

$KB \vDash \neg P_{1,1}$ (pasti tidak ada lubang di kotak [1,1])

$KB \nvDash P_{1,1}$ (tidak bisa disimpulkan bahwa ada lubang di [1,1])

In [46]:
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)

(False, False)

### Forward and backward chaining
Sebelumnya, kita sudah membahas dua algoritma untuk mengecek apakah sebuah kalimat entailed oleh KB. Dan kali ini akan ditambahkan algoritma ketiga.

Perbedaan utamanya adalah: tujuan kita sekarang adalah menentukan apakah sebuah basis pengetahuan (*knowledge base*) yang berisi definite clauses meng-*entail* sebuah simbol proposisi tunggal `q` (*query*).
Tapi ada satu catatan: basis pengetahuan di sini hanya boleh berisi Horn clauses.

Previously, we said we will look at two algorithms to check if a sentence is entailed by the `KB`. Here's a third one.
The difference here is that our goal now is to determine if a knowledge base of definite clauses entails a single proposition symbol *q* - the query.
There is a catch however - the knowledge base can only contain **Horn clauses**.
<br>
#### Horn Clauses
Horn clause dapat didefinisikan sebagai *disjungsi* dari *literal* dengan **paling banyak** satu literal positif.


-  Horn clause dengan tepat satu literal positif disebut *definite clause*
-  Contoh Horn clause:

$$\neg a\lor\neg b\lor\neg c\lor\neg d... \lor z$$

Ini, secara kebetulan, juga merupakan definite clause
<br>

Dengan menggunakan hukum De Morgan, contoh di atas bisa disederhanakan menjadi:
<br>
$$a\land b\land c\land d ... \implies z$$
<br>

Ini terlihat seperti reprsentasi logis dari cara manusia memproses data dan fakta, iya kan? Jika kita mengasumsikan bahwa `a`, `b`, `c`, `d` ... semuanya benar secara bersamaan, maka kita bisa menyimpulkan bahwa `z` juga benar pada saat itu.

Ada beberapa hal menarik dari *Horn clauses*, yang membuat inferensi algoritmik atau *resolution* menjadi lebih mudah:

- *Definite clause* dapat ditulis sebagai implikasi:

Sederhananya, sebuah *definite clause* bisa dipandang sebagai implikasi.
- - Premis (pengetahuan yang mendasari implikasi) adalah konjungsi literal positif
- - konklusinya adalah literal positif. Jadi kalimatnya menjadi lebih mudah dipahami. Premis biasannya disebut *body* dan kesimpulan disebut *head*. Sementara, sebuah literal positif tunggal disebut *fact*

- **Forward chaining dan Backward chaining** dapat digunakan untuk inferensi dengan **Horn clauses**: *Forward chaining* dan secara semantik identik dengan `AND-OR-Graph-Search` (Sebelumnya sudah dibahas di bab *search algorithms*). Detail implementasi akan dibahas sebentar lagi

- Memutuskan *entailment* dengan *Horn clauses* bersifat linear terhadap ukuran `KB`: Surprisingly, algoritma `forward and backward chaining` hanya menelusuri setiap elemen dalam `KB` paling banyak satu kali, sehingga masalah menjadi jauh lebih sederhana.



Funsi `pl_fc_entails` mengimplementasikan *forward chaining* untuk mengecek apakah sebuah `KB` meng-entail simbol `q`

Sebelum dilanjut, perlu dicatat bahwa `pl_fc_entails` tidak menggunakan `KB` biasa. Basis pengetahuan disini adalah instance dari kelas `PropDefiniteKB` (turunan dari `PropKB`) yang dimodifikasi untuk menyimpan *definite clauses*

Perbedaan utamanya adalah penambahan metode baru (`helper method`) pada `PropDefiniteKB` untuk mengembalikan daftar klausa dalam `KB` yang memiliki simbol `p` pada premisnya:

In [47]:
psource(PropDefiniteKB.clauses_with_premise)

Sekarang kita lihat algoritma `p1_fc_entailse`

In [48]:
psource(pl_fc_entails)

Fungsi ini menerima basis pengetahuan `KB` (instance dari `PropDefiniteKB`) dan sebuah query `q` sebagai input.

- `count` menyimpan jumlah simbol dalam premis setiap kalimat di `KB`
- fungsi bantu `conjuncts` memecah kalimat berdasarkan konjungsi
- `inferred` adalah `defaultdict` boolean untuk melacak apakah semua premis dari setiap klausa sudah diiferensikan
- `agenda` awalnya menyimpan daftar klausa yang sudah diketahui benar oleh `KB`. Fungsi `is_prop_symbol` mengecek apakah sebuah simbol adalah simbol logika proposisional yang valid.


selanjutnya, kita iterasi melalui `agenda`, setiap kali mengambil simbol `p`

- Jika query `q` sama dengan `p`, berarti *entailment* berlaku → kembalikan `True`.
- Agenda diproses dengan mengurangi `count` untuk setiap implikasi yang memiliki premis `p`.
- Jika `count` mencapai nol, artinya semua premis dari implikasi tersebut sudah diketahui benar, sehingga kesimpulannya bisa dimasukkan ke dalam agenda

Metode `clauses_with_premise` tadi digunakan untuk mencari klausa-klausa dalam `KB` yang memiliki `p` di premisnya

Sekarang kita sudah paham cara kerjanya. Selanjutnya, mari lihat contoh penggunaannya, tapi sebelumnya kita perlu mendefinisikan basis pengetahuan kita terlebih dahulu.

In [49]:
clauses = ['(B & F)==>E',
           '(A & E & F)==>G',
           '(B & C)==>F',
           '(A & B)==>D',
           '(E & F)==>H',
           '(H & I)==>J',
           'A',
           'B',
           'C']

# asumsikan kita mengetahui klausa klausa berikut bernilai benar

In [50]:
# sekarang dari semua klausa diatas kita akan `tell` ke basis pengetahuan (`KB`) kita
definite_clauses_KB = PropDefiniteKB()
for clause in clauses:
    definite_clauses_KB.tell(expr(clause))

# setelah itu, kita bisa mengecek apakah basis pengetahuan kita *meng-entail* query tertentu

In [51]:
pl_fc_entails(definite_clauses_KB, expr('G'))

True

1. kita mulai Dari fakta A, B, C

2. Karena ada B & C, maka dari klausa (3) kita dapat F

3. Sekarang kita punya B & F, dari klausa (1) kita dapat E

4. Jadi saat ini fakta yang kita tahu: A, B, C, F, E

5. Perhatikan klausa (2): (A & E & F) ==> G

* A sudah True

* E sudah True

* F sudah True

**Maka G bisa disimpulkan benar**

In [52]:
pl_fc_entails(definite_clauses_KB, expr('H'))

True

In [53]:
pl_fc_entails(definite_clauses_KB, expr('I'))

False

In [54]:
pl_fc_entails(definite_clauses_KB, expr('J'))

False

### Effective Propositional Model Checking

Bagian sebelumnya kita sudah membahas prosedur algoritmik untuk model checking.
Di bagian ini, kita akan melihat cara membuatnya lebih efisien secara komputasi

Masalah yang kita coba selesaikan biasanya disebut *propositional satisfiability problem* atau **SAT problem**
sederhananya, jika ada sebuah model yang dapat memuaskan suatu formula Boolean, maka formula tersebut disebut satisfiable.

Masalah SAT adalah masalah pertama yang terbukti sebagai *NP-complete*.
Karakteristik utama dari masalah *NP-complete* adalah:

- Jika ada solusi, mudah untuk memverifikasi bahwa solusi tersebut benar.

- Tetapi, waktu yang diperlukan untuk menemukan solusi dengan algoritma yang diketahui saat ini akan meningkat secara eksponensial terhadap ukuran masalah.


Karena sifat ini, metode **heuristik** dan pendekatan kira-kira sering digunakan untuk menemukan solusi masalah SAT.

Kemampuan menyelesaikan masalah SAT berskala besar sangat penting, karena banyak masalah kombinatorial dalam ilmu komputer bisa direduksi menjadi masalah SAT, yaitu pengecekan apakah suatu kalimat proposisional *satisfiable* dengan batasan tertentu.

dua algoritma baru yang dapat melakukan propositional model checking dengan cara yang lebih efisien. Akan diperkenalkan



#### 1. Algoritma DPLL (Davis-Putnam-Logeman-Loveland)

Algoritma ini mirip dengan Backtracking-Search,
tetapi bekerja secara rekursif dengan enumerasi setiap model secara depth-first dan memiliki beberapa perbaikan dibanding algoritma seperti `tt_entails`:

This algorithm is very similar to Backtracking-Search.
It recursively enumerates possible models in a depth-first fashion with the following improvements over algorithms like `tt_entails`:
1. Early termination:

Dalam beberapa kasus, algoritma bisa menentukan nilai kebenaran hanya dari *partial model*. Contohnya, $(P\lor Q)\land(P\lor R)$ pasti bernilai `true` jika $P$ bernilai `true`, terlepas dari nilai variabel lain.
Hal ini membuat ruang pencarian menjadi lebih kecil

2. Pure symbol heuristic:

Jika sebuah simbol selalu muncul dengan tanda yang sama (hanya positif atau negatif) di semua klausa, maka disebut sebagai *pure symbol*.
Supaya kalimat proposional `true`, pure symbol bisa langsung ditetapkan sesuai kebutuhan. Contoh: $(P\lor\neg Q)\land(\neg Q\lor\neg R)\land(R\lor P)$ Simbol *pure* disini adalah $P$ dan $Q$.

Maka, agar kalimat bernilai benar, $P = true$  dan $Q = false$

> Pure symbol heuristic membantu menyederhanakan masalah SAT sedikit

3. Unit clause heuristic:

Didalam konteks DPLL, klausa dengan hanya satu literal atau klausa yang semua literalnya bernilai `false` kecuali satu, disebut *unit clause*. Jika sebuah klausa adalah *unit clause*. Maka satu satunya cara agar klausa tersebut bernilai `true` adalah dengan memberi nilai benar pada literal terakhir,
Satu-satunya cara membuat klausa ini benar adalah dengan menetapkan literal tersebut ke nilai yang benar.

 *We have no other choice*

Menetapkan nilai pada sebuah *unit clause* bisa menghasilkan *unit clause* lain. Contohnya, jika $P = false$, maka klausa $(P \lor Q)$ berubah menjadi *unit clause* yang memaksa $Q = true$


Rangkaian penetapan secara paksa yang dihasilkan dari unit clauses sebelumnya disebut *unit propagation*. Dengan cara ini, heuristik ini semakin menyederhanakan masalah.

Algoritma DPLL sebenarnya juga menggunakan beberapa trik lain untuk menangani masalah berskala besar.
Namun, trik tersebut tidak dibahas di notebook ini. (Dari sananya begitu, lihat Section 7.6 pada buku untuk detail lebih lanjut.)

*Let's have a look at the algorithm*

In [55]:
psource(dpll)

Algoritma ini menggungakan ide-ide yang sudah dijelaskan sebelumnya, untuk mengecek *satisfiability* dan sebuah kalimat dalam logika proposisional.

algoritma bekerja secara rekursif, memanggil dirinya sendiri dan menyederhanakan masalah pada setiap langkah. dengan memanfaatkan fungsi pembantu `find_pure_symbol` dan `find_unit_clause` untuk melaksanakan langkah ke-2 dan ke-3 sebagaimana telah dijelaskan sebelumnya.

Agar lebih praktis, biasanya kita pakai helper function dpll_satisfiable yang akan otomatis mengubah input menjadi bentuk *Conjunctive Normal Form* (CNF) lalu memanggil dpll.

In [56]:
psource(dpll_satisfiable)

> Let's see a few examples of usage.






In [63]:
A, B, C, D = expr('A, B, C, D')

In [64]:
dpll_satisfiable(A & B & ~C & D)

{A: True, D: True, B: True, C: False}

This is a simple case to highlight that the algorithm actually works.

In [65]:
dpll_satisfiable((A & B) | (C & ~A) | (B & ~D))

{D: False, B: True}

Jika sebuah simbol tidak muncul di hasil solusi, itu berarti nilai simbol tersebut tidak memengaruhi kebenaran model.

In [66]:
dpll_satisfiable(A |'<=>'| B)

{A: True, B: True}

In [70]:
dpll_satisfiable((A |'<=>'| B) |'==>'| (C & ~A))

{C: True, A: True, B: False}

In [62]:
dpll_satisfiable((A | (B & C)) |'<=>'| ((A | B) & (A | C)))

{A: True}

#### 2. Algoritma WalkSAT

Algoritma ini sangat mirip dengan *Hill Climbing*.
Pada setiap iterasi, algoritma memilih sebuah klausa yang tidak terpuaskan (*unsatisfied clause*) dan membalikkan (*flip*) sebuah simbol dalam klausa tersebut.
Ini mirip dengan menemukan *neighboring state* dalam algoritma `hill_climbing`.

Simbol yang akan dibalik ditentukan oleh sebuah fungsi evaluasi yang menghitung jumlah klausa yang unsatisfied.
Kadang, simbol juga dibalik secara acak untuk menghindari `local optima`. Sebuah keseimbangan halus antara keserakahan (*greediness*) dan keacakan (*randomness*) diperlukan. Alternatifnya, beberapa versi algoritma memulai ulang (*restart*) dengan penugasan acak baru sepenuhnya jika tidak ditemukan solusi dalam waktu lama, sebagai cara untuk keluar dari `local minima` dari jumlah klausa yang tidak terpuaskan.

> Let's have a look at the algorithm.

In [71]:
psource(WalkSAT)

Fungsi ini menerima tiga argumentThe function takes three arguments:

1. `clauses` yaitu klausa-klausa yang ingin kita penuhi

2. `p` untuk probabilitas untuk mengubah sebuah simbol secara acak.

3. `max_flips` yang merupakan jumlah maksimum pembalikan (`flips`) yang akan dijalankan algoritma. Jika setelah itu klausa-klausa masih belum puas, algoritma mengembalikan `None` untuk menandakan kegagalan.

Let's see a few examples of usage.

In [72]:
A, B, C, D = expr('A, B, C, D')

In [73]:
WalkSAT([A, B, ~C, D], 0.5, 100)

{A: True, D: True, B: True, C: False}

Ini adalah kasus sederhana untuk menunjukkan bahwa algoritma dapat converge

In [74]:
WalkSAT([A & B, A & C], 0.5, 100)

{A: True, B: True, C: True}

In [75]:
WalkSAT([A & B, C & D, C & B], 0.5, 100)

{A: True, D: True, B: True, C: True}

In [76]:
WalkSAT([A & B, C | D, ~(D | B)], 0.5, 1000)

Kasus ini tidak ada output karena `WalkSAT` tidak menemukan model di mana klausa-klausa tersebut benar.
Kita bisa menyelesaikan klausa-klausa ini untuk melihat bahwa mereka bersama-sama membentuk kontradiksi, dan oleh karena itu memang tidak seharusnya memiliki solusi.

Satu perbedaan penting antara algoritma ini dengan `dpll_satisfiable` adalah bahwa kedua algoritma menerima input dengan cara yang berbeda.

Agar `WalkSAT` bisa menerima kalimat lengkap sebagai input, kita bisa menulis sebuah fungsi pembantu yang mengonversi kalimat input ke dalam bentuk CNF, lalu memanggil `WalkSAT` dengan daftar konjungsi dari bentuk CNF tersebut.

In [ ]:
def WalkSAT_CNF(sentence, p=0.5, max_flips=10000):
    return WalkSAT(conjuncts(to_cnf(sentence)), 0, max_flips)

Sekarang kita bisa memanggil `WalkSAT_CNF` dan `DPLL_satisfiable` dengan argumen yang sama.

In [ ]:
WalkSAT_CNF((A & B) | (C & ~A) | (B & ~D), 0.5, 1000)

{A: True, B: True, C: False, D: True}

**Yeay Berhasil !!**

Perhatikan bahwa solusi yang dihasilkan oleh `WalkSAT` tidak menghilangkan variabel yang tidak berhubungan dengan kalimat.
Jika kalimat independen terhadap variabel tertentu, solusinya akan berisi nilai acak untuk variabel tersebut karena sifat stokastik dari algoritma.


Kita bisa bandingkan runtime `WalkSAT` dan `DPLL` untuk beberapa kasus. Kita akan menggunakan `%%timeit` untuk melakukan ini.



In [ ]:
sentence_1 = A |'<=>'| B
sentence_2 = (A & B) | (C & ~A) | (B & ~D)
sentence_3 = (A | (B & C)) |'<=>'| ((A | B) & (A | C))

In [ ]:
%%timeit
dpll_satisfiable(sentence_1)
dpll_satisfiable(sentence_2)
dpll_satisfiable(sentence_3)

1.55 ms ± 64.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit
WalkSAT_CNF(sentence_1)
WalkSAT_CNF(sentence_2)
WalkSAT_CNF(sentence_3)

1.02 ms ± 6.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Rata-rata, untuk kasus yang dapat diselesaikan, `WalkSAT` lebih cepat daripada `DPLL` karena untuk jumlah variabel yang kecil,
`WalkSAT` dapat mengurangi ruang pencarian secara signifikan.

Hasilnya bisa berbeda untuk kalimat dengan lebih banyak simbol.
Kalian boleh banget untuk bereksperimen dengan ini, untuk dapat memahami *trade-off* dari algoritma-algoritma ini dengan lebih baik.

# Baru sampe sini

### SATPlan

In this section we show how to make plans by logical inference. The basic idea is very simple. It includes the following three steps:
1. Constuct a sentence that includes:
    1. A colection of assertions about the initial state.
    2. The successor-state axioms for all the possible actions at each time up to some maximum time t.
    3. The assertion that the goal is achieved at time t.
2. Present the whole sentence to a SAT solver.
3. Assuming a model is found, extract from the model those variables that represent actions and are assigned true. Together they represent a plan to achieve the goals.


Lets have a look at the algorithm

In [ ]:
psource(SAT_plan)

Let's see few examples of its usage. First we define a transition and then call `SAT_plan`.

In [ ]:
transition = {'A': {'Left': 'A', 'Right': 'B'},
            'B': {'Left': 'A', 'Right': 'C'},
            'C': {'Left': 'B', 'Right': 'C'}}


print(SAT_plan('A', transition, 'C', 2))
print(SAT_plan('A', transition, 'B', 3))
print(SAT_plan('C', transition, 'A', 3))

None
['Right']
['Left', 'Left']


Let us do the same for another transition.

In [ ]:
transition = {(0, 0): {'Right': (0, 1), 'Down': (1, 0)},
            (0, 1): {'Left': (1, 0), 'Down': (1, 1)},
            (1, 0): {'Right': (1, 0), 'Up': (1, 0), 'Left': (1, 0), 'Down': (1, 0)},
            (1, 1): {'Left': (1, 0), 'Up': (0, 1)}}


print(SAT_plan((0, 0), transition, (1, 1), 4))

['Right', 'Down']


## First-Order Logic Knowledge Bases: `FolKB`

The class `FolKB` can be used to represent a knowledge base of First-order logic sentences. You would initialize and use it the same way as you would for `PropKB` except that the clauses are first-order definite clauses. We will see how to write such clauses to create a database and query them in the following sections.

## Criminal KB
In this section we create a `FolKB` based on the following paragraph.<br/>
<em>The law says that it is a crime for an American to sell weapons to hostile nations. The country Nono, an enemy of America, has some missiles, and all of its missiles were sold to it by Colonel West, who is American.</em><br/>
The first step is to extract the facts and convert them into first-order definite clauses. Extracting the facts from data alone is a challenging task. Fortunately, we have a small paragraph and can do extraction and conversion manually. We'll store the clauses in list aptly named `clauses`.

In [ ]:
clauses = []

<em>“... it is a crime for an American to sell weapons to hostile nations”</em><br/>
The keywords to look for here are 'crime', 'American', 'sell', 'weapon' and 'hostile'. We use predicate symbols to make meaning of them.

* `Criminal(x)`: `x` is a criminal
* `American(x)`: `x` is an American
* `Sells(x ,y, z)`: `x` sells `y` to `z`
* `Weapon(x)`: `x` is a weapon
* `Hostile(x)`: `x` is a hostile nation

Let us now combine them with appropriate variable naming to depict the meaning of the sentence. The criminal `x` is also the American `x` who sells weapon `y` to `z`, which is a hostile nation.

$\text{American}(x) \land \text{Weapon}(y) \land \text{Sells}(x, y, z) \land \text{Hostile}(z) \implies \text{Criminal} (x)$

In [ ]:
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))

<em>"The country Nono, an enemy of America"</em><br/>
We now know that Nono is an enemy of America. We represent these nations using the constant symbols `Nono` and `America`. the enemy relation is show using the predicate symbol `Enemy`.

$\text{Enemy}(\text{Nono}, \text{America})$

In [ ]:
clauses.append(expr("Enemy(Nono, America)"))

<em>"Nono ... has some missiles"</em><br/>
This states the existence of some missile which is owned by Nono. $\exists x \text{Owns}(\text{Nono}, x) \land \text{Missile}(x)$. We invoke existential instantiation to introduce a new constant `M1` which is the missile owned by Nono.

$\text{Owns}(\text{Nono}, \text{M1}), \text{Missile}(\text{M1})$

In [ ]:
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))

<em>"All of its missiles were sold to it by Colonel West"</em><br/>
If Nono owns something and it classifies as a missile, then it was sold to Nono by West.

$\text{Missile}(x) \land \text{Owns}(\text{Nono}, x) \implies \text{Sells}(\text{West}, x, \text{Nono})$

In [ ]:
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))

<em>"West, who is American"</em><br/>
West is an American.

$\text{American}(\text{West})$

In [ ]:
clauses.append(expr("American(West)"))

We also know, from our understanding of language, that missiles are weapons and that an enemy of America counts as “hostile”.

$\text{Missile}(x) \implies \text{Weapon}(x), \text{Enemy}(x, \text{America}) \implies \text{Hostile}(x)$

In [ ]:
clauses.append(expr("Missile(x) ==> Weapon(x)"))
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))

Now that we have converted the information into first-order definite clauses we can create our first-order logic knowledge base.

In [ ]:
crime_kb = FolKB(clauses)

The `subst` helper function substitutes variables with given values in first-order logic statements.
This will be useful in later algorithms.
It's implementation is quite simple and self-explanatory.

In [ ]:
psource(subst)

Here's an example of how `subst` can be used.

In [ ]:
subst({x: expr('Nono'), y: expr('M1')}, expr('Owns(x, y)'))

Owns(Nono, M1)

## Inference in First-Order Logic
In this section we look at a forward chaining and a backward chaining algorithm for `FolKB`. Both aforementioned algorithms rely on a process called <strong>unification</strong>, a key component of all first-order inference algorithms.

### Unification
We sometimes require finding substitutions that make different logical expressions look identical. This process, called unification, is done by the `unify` algorithm. It takes as input two sentences and returns a <em>unifier</em> for them if one exists. A unifier is a dictionary which stores the substitutions required to make the two sentences identical. It does so by recursively unifying the components of a sentence, where the unification of a variable symbol `var` with a constant symbol `Const` is the mapping `{var: Const}`. Let's look at a few examples.

In [ ]:
unify(expr('x'), 3)

{x: 3}

In [ ]:
unify(expr('A(x)'), expr('A(B)'))

{x: B}

In [ ]:
unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(y)'))

{x: Bella, y: Dobby}

In cases where there is no possible substitution that unifies the two sentences the function return `None`.

In [ ]:
print(unify(expr('Cat(x)'), expr('Dog(Dobby)')))

None


We also need to take care we do not unintentionally use the same variable name. Unify treats them as a single variable which prevents it from taking multiple value.

In [ ]:
print(unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(x)')))

None


### Forward Chaining Algorithm
We consider the simple forward-chaining algorithm presented in <em>Figure 9.3</em>. We look at each rule in the knowledge base and see if the premises can be satisfied. This is done by finding a substitution which unifies each of the premise with a clause in the `KB`. If we are able to unify the premises, the conclusion (with the corresponding substitution) is added to the `KB`. This inferencing process is repeated until either the query can be answered or till no new sentences can be added. We test if the newly added clause unifies with the query in which case the substitution yielded by `unify` is an answer to the query. If we run out of sentences to infer, this means the query was a failure.

The function `fol_fc_ask` is a generator which yields all substitutions which validate the query.

In [ ]:
psource(fol_fc_ask)

Let's find out all the hostile nations. Note that we only told the `KB` that Nono was an enemy of America, not that it was hostile.

In [ ]:
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}]


The generator returned a single substitution which says that Nono is a hostile nation. See how after adding another enemy nation the generator returns two substitutions.

In [ ]:
crime_kb.tell(expr('Enemy(JaJa, America)'))
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}, {x: JaJa}]


<strong><em>Note</em>:</strong> `fol_fc_ask` makes changes to the `KB` by adding sentences to it.

### Backward Chaining Algorithm
This algorithm works backward from the goal, chaining through rules to find known facts that support the proof. Suppose `goal` is the query we want to find the substitution for. We find rules of the form $\text{lhs} \implies \text{goal}$ in the `KB` and try to prove `lhs`. There may be multiple clauses in the `KB` which give multiple `lhs`. It is sufficient to prove only one of these. But to prove a `lhs` all the conjuncts in the `lhs` of the clause must be proved. This makes it similar to <em>And/Or</em> search.

#### OR
The <em>OR</em> part of the algorithm comes from our choice to select any clause of the form $\text{lhs} \implies \text{goal}$. Looking at all rules's `lhs` whose `rhs` unify with the `goal`, we yield a substitution which proves all the conjuncts in the `lhs`. We use `parse_definite_clause` to attain `lhs` and `rhs` from a clause of the form $\text{lhs} \implies \text{rhs}$. For atomic facts the `lhs` is an empty list.

In [ ]:
psource(fol_bc_or)

#### AND
The <em>AND</em> corresponds to proving all the conjuncts in the `lhs`. We need to find a substitution which proves each <em>and</em> every clause in the list of conjuncts.

In [ ]:
psource(fol_bc_and)

Now the main function `fl_bc_ask` calls `fol_bc_or` with substitution initialized as empty. The `ask` method of `FolKB` uses `fol_bc_ask` and fetches the first substitution returned by the generator to answer query. Let's query the knowledge base we created from `clauses` to find hostile nations.

In [ ]:
# Rebuild KB because running fol_fc_ask would add new facts to the KB
crime_kb = FolKB(clauses)

In [ ]:
crime_kb.ask(expr('Hostile(x)'))

{v_5: x, x: Nono}

You may notice some new variables in the substitution. They are introduced to standardize the variable names to prevent naming problems as discussed in the [Unification section](#Unification)

## Appendix: The Implementation of `|'==>'|`

Consider the `Expr` formed by this syntax:

In [ ]:
P |'==>'| ~Q

(P ==> ~Q)

What is the funny `|'==>'|` syntax? The trick is that "`|`" is just the regular Python or-operator, and so is exactly equivalent to this:

In [ ]:
(P | '==>') | ~Q

(P ==> ~Q)

In other words, there are two applications of or-operators. Here's the first one:

In [ ]:
P | '==>'

PartialExpr('==>', P)

What is going on here is that the `__or__` method of `Expr` serves a dual purpose. If the right-hand-side is another `Expr` (or a number), then the result is an `Expr`, as in `(P | Q)`. But if the right-hand-side is a string, then the string is taken to be an operator, and we create a node in the abstract syntax tree corresponding to a partially-filled  `Expr`, one where we know the left-hand-side is `P` and the operator is `==>`, but we don't yet know the right-hand-side.

The `PartialExpr` class has an `__or__` method that says to create an `Expr` node with the right-hand-side filled in. Here we can see the combination of the `PartialExpr` with `Q` to create a complete `Expr`:

In [ ]:
partial = PartialExpr('==>', P)
partial | ~Q

(P ==> ~Q)

This  [trick](http://code.activestate.com/recipes/384122-infix-operators/) is due to [Ferdinand Jamitzky](http://code.activestate.com/recipes/users/98863/), with a modification by [C. G. Vedant](https://github.com/Chipe1),
who suggested using a string inside the or-bars.

## Appendix: The Implementation of `expr`

How does `expr` parse a string into an `Expr`? It turns out there are two tricks (besides the Jamitzky/Vedant trick):

1. We do a string substitution, replacing "`==>`" with "`|'==>'|`" (and likewise for other operators).
2. We `eval` the resulting string in an environment in which every identifier
is bound to a symbol with that identifier as the `op`.

In other words,

In [ ]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

is equivalent to doing:

In [ ]:
P, Q = symbols('P, Q')
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

One thing to beware of: this puts `==>` at the same precedence level as `"|"`, which is not quite right. For example, we get this:

In [ ]:
P & Q  |'==>'|  P | Q

(((P & Q) ==> P) | Q)

which is probably not what we meant; when in doubt, put in extra parens:

In [ ]:
(P & Q)  |'==>'|  (P | Q)

((P & Q) ==> (P | Q))

## Examples

In [ ]:
from notebook import Canvas_fol_bc_ask
canvas_bc_ask = Canvas_fol_bc_ask('canvas_bc_ask', crime_kb, expr('Criminal(x)'))

# Authors

This notebook by [Chirag Vartak](https://github.com/chiragvartak) and [Peter Norvig](https://github.com/norvig).

